#**TP1: Multilayer Perceptron**

### **Introdução**
Nesta atividade de programação, você implementará o Multilayer Perceptron (MLP) em Python usando apenas a biblioteca numpy. Esta implementação permitirá a criação de redes neurais com qualquer número de camadas. Este modelo será usado para resolver o problema de classificação de imagens de gatos. Ao final desta atividade, você terá uma rede neural profunda capaz de classificar se uma dada imagem é um gato ou não.

### **Objetivo**

O principal objetivo deste projeto é aprender a implementação do algoritmo MLP com um número flexível de camadas, principalmente a **Propagação das entradas** e a **Retropropagação dos erros**. Além disso, você implementará a função de ativação ReLU e ganhará experiência na comparação de modelos neurais de diferentes tamanhos.

### **Instruções**

As células onde você precisa escrever o código são destacadas com os seguintes comentários:

```python
### SEU CÓDIGO COMEÇA AQUI ### ≈x linhas
### SEU CÓDIGO TERMINA AQUI ###
```

Escreva suas soluções apenas entre estes dois comentários. Observe que o comentário inicial dá uma ideia do número de linhas de código esperado na solução.

Após cada célula deste tipo, haverá uma célula de teste seguida pelos resultados esperados, para que você possa saber se sua solução está correta.

### **Parte 0: Importar bibliotecas**

Nossa primeira tarefa é carregar a biblioteca numpy e outras bibliotecas auxiliares, que nos ajudarão a visualizar os dados e os resultados:

- **numpy**: principal biblioteca de computação científica em Python.
- **matplotlib**: principal biblioteca para plotagem de gráficos em Python.
- **h5py**: ler conjuntos de dados no formato h5.
- **PIL**: testar seu modelo com suas próprias imagens.

**Observação**:
Como visto em aula, precisamos inicializar os pesos MLP com valores aleatórios próximos de zero. Portanto, neste projeto, inicializamos o gerador de números aleatórios numpy (np.random.seed(1)) com uma semente fixa, para que seus resultados sejam iguais aos esperados.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image

%matplotlib inline

np.random.seed(1)

### **Parte 1: Carregar e pré-processar o conjunto de dados**

#### **1.1 Baixar o conjunto de dados**

O conjunto de dados deste projeto contém:

- Treinamento: $n_{tr}$ imagens rotuladas como Gato ($y=1$) ou Não-gato ($y=0$)
- Teste: $n_{te}$ imagens rotuladas como Gato ($y=1$) ou Não-gato ($y=0$)

Todas as imagens são quadradas e coloridas. Portanto, elas são representadas por matrizes numpy do formato `(num_px, num_px, 3)`, onde `num_px` é a largura e a altura da imagem e 3 se refere aos canais de cor (RGB - Red, Green, Blue).

In [ ]:
!wget -O 'train_catvnoncat.h5' 'https://lucasnfe.github.io/ufv-inf721/static_files/datasets/p1-regressao-logistica/train_catvnoncat.h5'
!wget -O 'test_catvnoncat.h5' 'https://lucasnfe.github.io/ufv-inf721/static_files/datasets/p1-regressao-logistica/test_catvnoncat.h5'

train_dataset = h5py.File('train_catvnoncat.h5', "r")
train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # imagens do conjunto de treino
train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # rótulos do conjunto de treino

test_dataset = h5py.File('test_catvnoncat.h5', "r")
test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # imagens do conjunto de teste
test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # rótulos do conjunto de teste

classes = np.array([b"nao gato", b"gato"]) # lista das classes, classe 0 -> 'não gato'; classe 1 -> 'gato'

train_set_y = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
test_set_y = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

#### **1.2 Visualização de exemplos de conjuntos de dados**

Cada linha de `train_set_x_orig` e `test_set_x_orig` é uma matriz numpy que representa uma imagem. Você pode visualizar um exemplo executando o código a seguir. Sinta-se à vontade para alterar o valor da variável `index` e executar novamente para ver outras imagens.

In [ ]:
# visualizando uma imagem do conjunto de treinamento
index = 13
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[0, index]) + ", é uma imagem de '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "'.")

#### **1.3 Dimensionalidade do problema**

Esta é a primeira célula onde você escreverá o código. Crie as seguintes variáveis:

- `n_train` (número de exemplos de treinamento)
- `n_test` (número de exemplos de teste)
- `num_px` (altura e largura de uma imagem do conjunto de dados)

Use a propriedade `shape` de matrizes numpy e lembre-se de que `train_set_x_orig` é uma matriz numpy com o formato `(m_train, num_px, num_px, 3)`.

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### (≈3 linhas)

### SEU CÓDIGO TERMINA AQUI ###

print ("Number of training examples: n_train = " + str(n_train))
print ("Number of testing examples: n_test = " + str(n_test))
print ("Height/Width of the images: num_px = " + str(num_px))
print("---")
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

**Resultado Esperado**:
<table style="width:15%">
  <tr>
    <td><b>n_train<b></td>
    <td>209</td>
  </tr>
  
  <tr>
    <td><b>n_test<b></td>
    <td> 50 </td>
  </tr>
  
  <tr>
    <td><b>num_px<b></td>
    <td> 64 </td>
  </tr>
  
</table>

#### **1.4 Pré-processamento dos dados**

#### **1.4.1 Transformando imagens em vetores de características**

Para processar as imagens do conjunto de dados com regressão logística, precisamos transformá-las em vetores de características. Para isso, achataremos essas imagens, originalmente com o formato `(num_px, num_px, 3)`, em vetores com o formato `(num_px * num_px * 3, 1)`.


Usaremos a função reshape dos arrays numpy para modificar o formato das imagens de treinamento `(train_set_x_orig)` e de teste `(test_set_x_orig)` e, então, armazenaremos as imagens de treinamento e teste achatadas em novos arrays chamados `train_set_x_flatten` e `test_set_x_flatten`, respectivamente.

In [ ]:
train_set_x_flatten = train_set_x_orig.reshape((n_train, -1)).T
test_set_x_flatten = test_set_x_orig.reshape((n_test, -1)).T

print ("train_set_x_flatten tamanho: " + str(train_set_x_flatten.shape))
print ("train_set_y tamanho: " + str(train_set_y.shape))
print ("test_set_x_flatten tamanho: " + str(test_set_x_flatten.shape))
print ("test_set_y tamanho: " + str(test_set_y.shape))
print ("Primeiras 5 características (pixel values) do primeiro exemplo de treino: " + str(train_set_x_flatten[0:5,0]))

**Resultado Esperado**:

<table style="width:35%">
  <tr>
    <td><b> train_set_x_flatten shape<b></td>
    <td> (12288, 209)</td>
  </tr>
  <tr>
    <td><b>train_set_y shape<b></td>
    <td>(1, 209)</td>
  </tr>
  <tr>
    <td><b> test_set_x_flatten shape<b></td>
    <td>(12288, 50)</td>
  </tr>
  <tr>
    <td><b>test_set_y shape<b></td>
    <td>(1, 50)</td>
  </tr>
  <tr>
  <td>First 5 features (pixel values) of the first training example</td>
  <td>[17 31 56 22 33]</td>
  </tr>
</table>

#### **1.4.2 Normalizar valores de pixel**

Uma etapa muito comum de pré-processamento de imagens em aprendizado de máquina é normalizar valores de pixel entre 0 e 1. Para isso, basta dividir cada linha do conjunto de dados por 255 (o valor máximo de um canal de pixel). Divida as matrizes `train_set_x_flatten` e `test_set_x_flatten` de imagens achatadas por 255 e armazene os resultados em novas matrizes chamadas `train_set_x` e `test_set_x`.

In [ ]:
### SEU CÓDIGO COMEÇA AQUI ### (≈2 linhas)

### SEU CÓDIGO TERMINA AQUI ###

print ("First 5 features of the normalized feature vector of the first training example: " + str(train_set_x[0:5,0]))

**Resultado Esperado**:

<table style="width:35%">
  <tr>
    <td>First 5 features of the normalized feature vector of the first training example</td>
    <td> [0.06666667 0.12156863 0.21960784 0.08627451 0.12941176]</td>
  </tr>
</table>

### **Parte 2: Definir o modelo**

Neste projeto, usaremos o MLP para a classificação de imagens de gatos. Lembre-se de que o MLP organiza os neurônios em uma estrutura em camadas. A formulação geral do MLP permite que tanto o número de camadas L quanto o número de neurônios $m^{[l]}$ por camada $1 \leq l \leq L$ sejam configuráveis ​​de acordo com a complexidade do problema de aprendizado. Além disso, as funções de ativação dos neurônios do MLP não precisam necessariamente ser a função sigmoide. Podemos escolher a função de ativação para cada camada da rede. Lembre-se de que esta formulação geral do MLP tem a seguinte hipótese:

$$
\begin{align}
Z^{[l]} &= W^{[l]} A^{[l-1]} + \mathbf{b}^{[l]} \\
\mathbf{A}^{[l]} &= g^{[l]}(Z^{[l]}) \\
\end{align} $$, onde:

- $W^{[l]}$ é a matriz de pesos da camada $l$ com dimensões $({m^{[l]}, m^{[l-1]}}$);
- $\mathbf{b}^{[l]}$ é o vetor de polarização da camada $l$ com dimensões $(m^{[l]}, 1)$;
- $A^{[L]}$ é a matriz de ativação da camada $l$ com dimensões $(m^{[l]}, n)$;
- $g^{[l]}$ é a função de ativação da camada $l$.

De acordo com esta definição, teremos as seguintes dimensões para uma rede com L camadas (Lembre-se de que $A^{[0]} = X$ e $A^{[L]} = \hat{Y}$):

<table style="width:100%">
    <tr>
        <td>  </td>
        <td> <b>  Dimensão de $W^{[l]}$  <b> </td>
        <td> <b>  Dimensão de $b^{[l]}$  <b>  </td>
        <td> <b>  Dimensão de $A^{[l]}$  <b> </td>
    <tr>
    <tr>
        <td> <b> Camada [1] <b> </td>
        <td> $(m^{[1]}, 12288)$ </td>
        <td> $(m^{[1]}, 1)$ </td>
        <td> $(m^{[1]}, 209)$ </td>
    <tr>
    <tr>
        <td> <b> Camada [2] <b> </td>
        <td> $(m^{[2]}, m^{[1]})$  </td>
        <td> $(m^{[2]}, 1)$ </td>
        <td> $(m^{[2]}, 209)$ </td>
    <tr>
       <tr>
        <td> $\vdots$ </td>
        <td> $\vdots$  </td>
        <td> $\vdots$  </td>
        <td> $\vdots$  </td>
    <tr>
   <tr>
        <td> <b> Camada [L-1] <b> </td>
        <td> $(m^{[L-1]}, m^{[L-2]})$ </td>
        <td> $(m^{[L-1]}, 1)$  </td>
        <td> $(m^{[L-1]}, 209)$ </td>
    <tr>
   <tr>
        <td> <b> Camada [L] <b> </td>
        <td> $(m^{[L]}, m^{[L-1]})$ </td>
        <td> $(m^{[L]}, 1)$ </td>
        <td> $(m^{[L]}, 209)$  </td>
    <tr>
</table>

#### **2.1 Funções de Ativação**

#### **2.1.1 Sigmoide**

O problema de aprendizado neste projeto é classificação binária. Portanto, usaremos a função de ativação sigmoide na camada de saída e a função de perda de Entropia Cruzada Binária (*Binary Cross-Entropy*). Dessa forma, implemente abaixo a função sigmoide. É importante ressaltar que além da ativação, a função também retorna o valor de Z, que chamamos de "cache", pois usaremos esse valor durante o cálculo do gradiente durante o *backpropagation*.

In [ ]:
def sigmoid(Z):
    """
    Implementa a ativação sigmoide em numpy

    Argumentos:
    Z -- array numpy de qualquer formato

    Retorna:
    A -- saída de sigmoide(z), mesmo formato que Z
    cache -- retorna Z também, útil durante o backpropagation
    """
    ### SEU CÓDIGO COMEÇA AQUI ### ≈1 linha

    ### SEU CÓDIGO TERMINA AQUI ###
    cache = Z

    return A, cache

#### **2.1.2 Relu**

Como visto em aula, ReLU é atualmente uma das escolhas mais populares para a função de ativação de neurônios em camadas ocultas. Neste projeto, também usaremos essa função nessas camadas.

Implemente a função de ativação ReLU. Observe que, assim como a sigmoide, a função retorna tanto a ativação quanto o valor de Z. Lembre-se de que ReLU é definido por $relu(z) = max(0, z)$. Em numpy, a função max é implementada por `np.maximum`.

In [ ]:
def relu(Z):
    """
    Implemente a função RELU.

    Argumentos:
    Z -- Saída da camada linear, de qualquer formato

    Retorna:
    A -- Parâmetro pós-ativação, do mesmo formato que Z
    cache -- um dicionário Python contendo "A"; armazenado para calcular o backpropagation de forma eficiente
    """
    ### SEU CÓDIGO COMEÇA AQUI ### ≈1 linha

    ### SEU CÓDIGO TERMINA AQUI ###
    assert(A.shape == Z.shape)

    cache = Z
    return A, cache

In [ ]:
Z = np.array([[-2.0 , 7.0]])
A, cache = relu(Z)

print("Ativação: ", A)
print("Cache: ", cache)

**Resultados Esperados:**
<table style="width:50%">
  <tr>
    <td><b> Ativação </b></td>
    <td > [[ 0.  7.]]</td>
  </tr>
  <tr>
    <td><b> Cache </b></td>
    <td > [[ -2.  7.]] </td>
  </tr>
</table>

### **2.2 - Multilayer Perceptron (MLP)**
Implemente a propagação direta de um MLP com L camadas. Para isso, implemente duas funções auxiliares: `linear_forward` e `linear_activation_forward`. A primeira calcula a parte linear $Z^{[l]}$ e a segunda calcula a ativação $g^{[l]}(Z^{[l]})$ da camada $l$. Isso facilitará a implementação do *backpropagation* nas próximas etapas.

#### **2.1 Combinação Linear da Camada $l$**

Implemente a combinação linear $Z^{[l]}$ entre os pesos e as entradas de uma camada $l$ do MLP. Lembre-se de que essa combinação é dada pela equação $Z^{[l]} = W^{[l]} A^{[l-1]} + \mathbf{b}^{[l]}$.

In [ ]:
def linear_forward(A, W, b):
    """
    Implementa a parte linear da propagação direta de uma camada.

    Argumentos:
    A -- ativações da camada anterior (ou dados de entrada): (tamanho da camada anterior, número de exemplos)
    W -- matriz de pesos: array numpy de tamanho (tamanho da camada atual, tamanho da camada anterior)
    b -- vetor de viés, array numpy de tamanho (tamanho da camada atual, 1)

    Retorna:
    Z -- a entrada da função de ativação, também chamada de parâmetro de pré-ativação
    cache -- um dicionário Python contendo "A", "W" e "b"; armazenado para calcular o backpropagation de forma eficiente
    """

    ### SEU CÓDIGO COMEÇA AQUI ### ≈1 linha

    ### SEU CÓDIGO TERMINA AQUI ###

    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)

    return Z, cache

In [ ]:
np.random.seed(1)

A = np.random.randn(3,2)
W = np.random.randn(1,3)
b = np.random.randn(1,1)
Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

**Resultados Esperados:**

<table style="width:35%">
  
  <tr>
    <td><b> Z </b></td>
    <td> [[ 3.26295337 -1.23429987]] </td>
  </tr>
  
</table>

#### **2.2 Ativação da Camada $l$**

Implemente a ativação $g^{[l]}(Z^{[l]})$ da camada $l$ do MLP. Embora a função de ativação ReLU em camadas ocultas torne o processo de aprendizado mais rápido do que a sigmoide, também suportaremos o uso de sigmoide nessas camadas, para que possamos conduzir experimentos com diferentes ativações. Lembre-se de que essa combinação é dada pela equação $g^{[l]}(Z^{[l]})$,
onde $Z^{[l]}$ é calculado pela função `linear_forward` implementada na etapa anterior e $g^{[l]}$ pode ser `sigmoide` ou `relu`, dependendo
do parâmetro `activation` (string).

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implementa a propagação direta para a camada LINEAR->ATIVAÇÃO

    Argumentos:
    A_prev -- ativações da camada anterior (ou dados de entrada): (tamanho da camada anterior, número de exemplos)
    W -- matriz de pesos: matriz numpy de tamanho (tamanho da camada atual, tamanho da camada anterior)
    b -- vetor de polarização, matriz numpy de tamanho (tamanho da camada atual, 1)
    activation -- a ativação a ser usada nesta camada, armazenada como uma string de texto: "sigmoid" ou "relu"

    Retorna:
    A -- a saída da função de ativação, também chamada de valor pós-ativação
    cache -- um dicionário Python contendo "linear_cache" e "activation_cache";
    armazenado para calcular a passagem reversa de forma eficiente
    """

    if activation == "sigmoid":
        # Entradas: "A_prev, W, b". Saídas: "A, activation_cache".
        ### SEU CÓDIGO COMEÇA AQUI ### ~2 linhas


        ### SEU CÓDIGO TERMINA AQUI ###


    elif activation == "relu":
        # Entradas: "A_prev, W, b". Saídas: "A, activation_cache".
        ### SEU CÓDIGO COMEÇA AQUI ### ~2 linhas


        ### SEU CÓDIGO TERMINA AQUI ###
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [ ]:
np.random.seed(2)
A_prev = np.random.randn(3,2); W = np.random.randn(1,3); b = np.random.randn(1,1)

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("Ativação (sigmoid) = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("Ativação (relu) = " + str(A))

**Resultados Esperados:**       
<table style="width:35%">
  <tr>
    <td> <b> Ativação (sigmoid) </b></td>
    <td > [[ 0.96890023  0.11013289]]</td>
  </tr>
  <tr>
    <td><b> Ativação (relu) </b></td>
    <td > [[ 3.43896131  0.        ]]</td>
  </tr>
</table>


### **2.3 Forward Propagation (Forward Pass)**

A função `linear_activation_forward` calcula a ativação de uma única camada $l$. Use esta função para implementar a função `forward_pass`, que realiza a propagação das entradas $X$ por todas as camadas da rede. Esquematicamente, a arquitetura do MLP com L camadas pode ser vista da seguinte forma:

$X$ $→$ (Linear | ReLU) $\times$ [L-1] $→$ (Linear | Sigmóide) $→$ $\hat{Y}$

Primeiramente, calcule a ativação $A^{[l]}$ para as [L-1] camadas ocultas (Linear $→$ ReLU) e, em seguida, calcule a ativação $A^{[L]} = \hat{Y}$ da camada de saída. Para calcular a ativação de uma camada $l$ com a função `linear_activation_forward`, você precisa dos pesos $W^{[l]}$ e $\mathbf{b}^{[l]}$ dessa camada. A variável `parameters` da função `forward_pass` armazena os pesos de todas as camadas em um dicionário. Para acessar os pesos de uma determinada camada $l$, basta criar uma chave de string com o nome do parâmetro e o número da camada. Por exemplo, `parameters['W1']` e `parameters['b1']` contêm os parâmetros $W^{[1]}$ e $\mathbf{b}^{[1]}$ da camada 1, respectivamente.

Lembre-se de que, além da ativação $A^{[l]}$, a função `linear_activation_forward` retorna o valor de $Z^{[l]}$ em um cache. Como precisaremos desses valores durante o backprop, armazene (`cache.append()`) as saídas de cada camada na lista `caches`.

In [ ]:
def forward_pass(X, parameters):
    """
    Implementar propagação direta para o cálculo [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID

    Argumentos:
    X -- dados, array numpy de tamanho (tamanho da entrada, número de exemplos)
    parâmetros -- saída de initialize_parameters_deep()

    Retornos:
    Y_hat -- último valor pós-ativação
    caches -- lista de caches contendo:
    todos os caches de linear_relu_forward() (há L-1 deles, indexados de 0 a L-2)
    o cache de linear_sigmoid_forward() (há um, indexado L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2 # numeros de camadas na rede neural

    # Implemente [LINEAR -> RELU]*(L-1). Adicione "cache" à lista "caches".
    for l in range(1, L):
        A_prev = A
        ### SEU CÓDIGO COMEÇA AQUI ### ~2 linhas


        ### SEU CÓDIGO TERMINA AQUI ###

    # Implemente LINEAR -> SIGMOID. Adicione "cache" à lista "caches".
    ### SEU CÓDIGO COMEÇA AQUI ### ~2 linhas


    ### SEU CÓDIGO TERMINA AQUI ###

    assert(Y_hat.shape == (1,X.shape[1]))

    return Y_hat, caches

In [ ]:
np.random.seed(6)
X = np.random.randn(5,4)
W1 = np.random.randn(4,5); b1 = np.random.randn(4,1)
W2 = np.random.randn(3,4); b2 = np.random.randn(3,1)
W3 = np.random.randn(1,3); b3 = np.random.randn(1,1)
parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2, "W3": W3, "b3": b3}

Y_hat, caches = forward_pass(X, parameters)
print("Y_hat = " + str(Y_hat))
print("Tamanho do cache = " + str(len(caches)))

**Resultados Esperados**       

<table style="width:50%">
  <tr>
    <td><b> Y_hat</b> </td>
    <td > [[ 0.03921668  0.70498921  0.19734387  0.04728177]]</td>
  </tr>
  <tr>
    <td> <b>Tamanho do cache </b> </td>
    <td > 3 </td>
  </tr>
</table>

### **Parte 3: Inicializar pesos**

Ao contrário da regressão logística, em MLP não podemos inicializar os pesos $W^{[l]}$ de uma camada $l$ com zeros, pois isso tornaria os neurônios na mesma camada idênticos. Em MLP, os pesos são inicializados com valores aleatórios próximos de zero.

Em aula, vimos uma solução simples que consistia em gerar uma matriz aleatória com valores entre 0 e 1 e multiplicá-la por uma constante pequena (por exemplo, 0,01). No entanto, uma solução melhor é gerar uma matriz aleatória com valores entre 0 e 1 e dividi-la pela raiz quadrada do número de entradas $m[^{[l-1]}]$ da camada $l$. Isso ajuda a garantir que a saída de um neurônio não seja dominada por um pequeno número de entradas, o que poderia levar a um sobreajuste. Quando um neurônio recebe um grande número de entradas, a saída pode ser altamente variável. Isso ocorre porque algumas entradas podem ter um impacto muito maior nos resultados do que outras. Por exemplo, se um neurônio recebe 100 entradas e uma delas é 10 vezes maior que as outras 99, essa entrada dominará a saída. Para evitar esse problema, os pesos das entradas são divididos pela raiz quadrada do número de entradas.

Implemente a seguinte função para inicializar os pesos MLP usando essa técnica. Para gerar uma matriz aleatória com valores entre 0 e 1, use a função `np.random.randn(m_l, m_{l-1})`. Para calcular a raiz quadrada do número de entradas de uma camada, use a função `np.sqrt(m_{l-1})`. Os pesos $b^{[l]}$ podem ser inicializados com zero. Para isso, use a função `np.zeros((m_l, 1))`. Observe que esta função recebe como parâmetro uma lista `layer_dims` onde um elemento com índice $l$ contém o número de neurônios na camada $l$.

In [ ]:
def initialize_parameters(layer_dims):
    """
    Argumentos:
    layer_dims -- array python (lista) contendo as dimensões de cada camada em nossa rede

    Retorna:
    parameters -- dicionário python contendo seus parâmetros "W1", "b1", ..., "WL", "bL":
    Wl -- matriz de pesos da forma (layer_dims[l], layer_dims[l-1])
    bl -- vetor de polarização da forma (layer_dims[l], 1)
    """

    np.random.seed(1)
    parameters = {}
    L = len(layer_dims) # número de camadas na rede

    for l in range(1, L):
        ### SEU CÓDIGO COMEÇA AQUI ### ~2 linhas


        ### SEU CÓDIGO TERMINA AQUI ###

        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

In [ ]:
parameters = initialize_parameters([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Resultados Esperados:**       
       
<table style="width:80%">
  <tr>
    <td><b>W1<b></td>
    <td>[[ 0.72642933 -0.27358579 -0.23620559 -0.47984616  0.38702206]
 [-1.0292794   0.78030354 -0.34042208  0.14267862 -0.11152182]
 [ 0.65387455 -0.92132293 -0.14418936 -0.17175433  0.50703711]
 [-0.49188633 -0.07711224 -0.39259022  0.01887856  0.26064289]]</td>
  </tr>
  
  <tr>
    <td><b>b1<b> </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]</td>
  </tr>
  
  <tr>
    <td><b>W2<b></td>
    <td>[[-0.55030959  0.57236185  0.45079536  0.25124717]
 [ 0.45042797 -0.34186393 -0.06144511 -0.46788472]
 [-0.13394404  0.26517773 -0.34583038 -0.19837676]]</td>
  </tr>
  
  <tr>
    <td><b>b2<b> </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]]</td>
  </tr>
  
</table>

### **Parte 4: Defina a função de perda**

Como temos um problema de classificação binária, usaremos a Entropia Cruzada Binária (*BCE*) como função de perda. Em MLP, o problema de otimização com esta função não é mais convexo, mas a descida do gradiente ainda funciona muito bem para otimizar os pesos da rede. Lembre-se de que a Entropia Cruzada Binária é definida da seguinte forma:

$$
L(h) = -\frac{1}{m}\sum_{i=1}^{m}(y_i\;log\ \hat{y_i} + (1 - y_i)\;log\;(1 - \hat{y_i})) \tag{7}
$$


In [ ]:
def binary_cross_entropy(Y_hat, Y):
    """
    Implemente a função de custo definida pela equação (7).

    Argumentos:
    Y_hat -- vetor de probabilidade correspondente às suas previsões de rótulo, forma (1, número de exemplos)
    Y -- vetor "rótulo" verdadeiro (por exemplo: contendo 0 se não for gato, 1 se gato), forma (1, número de exemplos)

    Retorna:
    custo -- custo de entropia cruzada
    """

    m = Y.shape[1]
    ### SEU CÓDIGO COMEÇA AQUI ### ~1 linha

    ### SEU CÓDIGO TERMINA AQUI ###

    loss = np.squeeze(loss) # Para garantir que o formato do seu custo seja o que esperamos (por exemplo, isso transforma [[17]] em 17).
    assert(loss.shape == ())

    return loss

In [ ]:
np.random.seed(1)
y_hat = np.random.randn(5, 1)
y = (np.random.randn(5, 1) > 0.3) * 1
print(y_hat)
print(y)
binary_cross_entropy(y_hat, y)

### **Parte 5: Retropropagação (Backpropagation)**

Os pesos de um MLP são atualizados com o algoritmo de Retropropagação.
Este algoritmo calcula eficientemente as derivadas parciais da função de erro em relação aos pesos da rede, aplicando a regra da cadeia de trás para frente, começando na camada de saída e retornando à camada de entrada. Como vimos em aula, cada nó em nosso grafo computacional precisa saber como calcular seus gradientes locais, que são as derivadas parciais de sua saída em relação às suas entradas. Nesta tarefa de programação, temos quatro nós em nosso grafo computacional: `linear(A, w, b)`, `sigmoid(z)`, `relu(z)` e `binary_cross_entropy(y_hat, y)`. Portanto, precisamos definir os gradientes locais desses quatro nós. Como já calculamos as derivadas parciais da perda de entropia cruzada binária para esses quatro nós em aula, calcularemos seus gradientes globais diretamente.

Primeiro, você implementará a função `linear_backward` para calcular as derivadas parciais da função de perda $L$ em relação aos pesos ($W^{[l]}$, $b^{[l]}$) e à ativação da camada anterior $A^{[l-1]}$. Em segundo lugar, as funções `sigmoid_backward` e `relu_backward` para calcular as derivadas das funções de ativação em relação às entradas Z. Em seguida, você implementará a função `linear_activation_backward`, que combina essas funções para calcular a retropropagação para uma única camada $l$.
Finalmente, você implementará a função `backward_pass`, que utiliza
a função `linear_activation_backward` para calcular a retropropagação em todas as camadas do MLP.

### **5.1 Derivadas Parciais da Combinação Linear da Camada $l$**

Derivadas parciais da função de perda $L$ em relação aos pesos ($W^{[l]}$, $b^{[l]}$) de uma camada $l$ do MLP:

$$ dW^{[l]} = \frac{\partial L}{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} $$
$$ db^{[l]} = \frac{\partial L}{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)} $$
$$ dA^{[l-1]} = \frac{\partial L}{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} $$

In [ ]:
def linear_backward(dZ, cache):
    """
    Implementa a porção linear da propagação para trás para uma única camada (camada l)

    Argumentos:
    dZ -- Gradiente do custo em relação à saída linear (da camada l atual)
    cache -- tupla de valores (A_prev, W, b) provenientes da propagação para frente na camada atual

    Retorna:
    dA_prev -- Gradiente do custo em relação à ativação (da camada l-1 anterior), mesmo formato que A_prev
    dW -- Gradiente do custo em relação a W (camada l atual), mesmo formato que W
    db -- Gradiente do custo em relação a b (camada l atual), mesmo formato que b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### SEU CÓDIGO COMEÇA AQUI ### ~3 linhas



    ### SEU CÓDIGO TERMINA AQUI ###

    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)

    return dA_prev, dW, db

In [ ]:
# cria algumas amostras de testes
np.random.seed(1)

dZ = np.random.randn(3,4)
A = np.random.randn(5,4)
W = np.random.randn(3,5)
b = np.random.randn(3,1)
linear_cache = (A, W, b)

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

**Resultados Esperados:**       
       
<table style="width:80%">
  <tr>
    <td> <b> dA_prev </b> </td>
    <td>[[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]</td>
  </tr>
  
  <tr>
    <td><b> dW </b> </td>
    <td>[[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]</td>
  </tr>
  
  <tr>
    <td><b> db </b></td>
    <td>[[-0.14713786]
 [-0.11313155]
 [-0.13209101]]</td>
  </tr>

  
</table>

### **5.2 Derivadas de funções de ativação**

Derivadas da função sigmoide em relação à entrada Z:

$$
\frac{\partial g}{\partial Z} = \frac{\partial L}{\partial a} \cdot (g(z) \cdot (1 - g(z))
$$

Caso tenha curiosidade, pode conferir como foi realizada a [derivação](https://math.stackexchange.com/a/1225116)

In [ ]:
def sigmoid_backward(dA, cache):
    """
    Implementa a propagação regressiva para uma única unidade SIGMOID.

    Argumentos:
    dA -- gradiente pós-ativação, de qualquer formato
    cache -- onde armazenamos 'Z' para calcular a propagação regressiva de forma eficiente

    Retorna:
    dZ -- Gradiente do custo em relação a Z
    """
    Z = cache

    g_z = 1/(1 + np.exp(-Z))
    ### SEU CÓDIGO COMEÇA AQUI ### ~1 linha

    ### YOUR CODE ENDS HERE ###

    assert (dZ.shape == Z.shape)

    return dZ

In [ ]:
np.random.seed(1)

dA = np.random.randn(3,4)
cache = np.random.randn(3,4)

dZ = sigmoid_backward(dA, cache)
print('dZ: ', dZ)

**Resultados Esperados:**       
       
<table style="width:80%">
  <tr>
    <td> <b> dZ </b> </td>
    <td>[[ 0.39571307 -0.14743535 -0.09728415 -0.20105294]
 [ 0.21475173 -0.47735759  0.43600867 -0.17501431]
 [ 0.05975979 -0.04567319  0.30026189 -0.48384433]]</td>
  </tr>
  
</table>

Derivada da função ReLU em relação à entrada Z:

$$
\frac{\partial g}{\partial Z} = \begin{cases}
0\, \text{se}\ z < 0 \\
1\, \text{se}\ z > 0 \\
\nexists, \text{se}\ z = 0 \\
\end{cases}
$$

Observe que, matematicamente, a derivada de ReLU é indefinida para $Z = 0$. Na prática, assumiremos que ela é igual a zero.

In [ ]:
def relu_backward(dA, cache):
    """
    Implementa a propagação regressiva para uma única unidade RELU.

    Argumentos:
    dA -- gradiente pós-ativação, de qualquer formato
    cache -- onde armazenamos 'Z' para calcular a propagação regressiva de forma eficiente

    Retorna:
    dZ -- Gradiente do custo em relação a Z
    """

    Z = cache
    dZ = np.array(dA, copy=True) # apenas convertendo dz para um objeto correto.

    # Quando z <= 0, você deve setar dz para 0 também
    ### SEU CÓDIGO COMEÇA AQUI ### ~1 linha

    ### SEU CÓDIGO TERMINA AQUI ###

    assert (dZ.shape == Z.shape)

    return dZ

In [ ]:
np.random.seed(1)

dA = np.random.randn(3,4)
cache = np.random.randn(3,4)

dZ = relu_backward(dA, cache)
print('dZ: ', dZ)

**Resultados Esperados:**       
       
<table style="width:80%">
  <tr>
    <td> <b> dZ </b> </td>
    <td>[[ 0.          0.         -0.52817175  0.        ]
 [ 0.          0.          1.74481176 -0.7612069 ]
 [ 0.         -0.24937038  1.46210794 -2.06014071]]</td>
  </tr>
  
</table>

### **5.3 Derivadas Parciais da Ativação da Camada $l$**

In [ ]:
def linear_activation_backward(dA, cache, activation):
    """
    Implementa a propagação regressiva para a camada LINEAR->ATIVAÇÃO.

    Argumentos:
    dA -- gradiente pós-ativação para a camada atual l
    cache -- tupla de valores (linear_cache, activation_cache) que armazenamos para calcular a propagação regressiva de forma eficiente
    activation -- a ativação a ser usada nesta camada, armazenada como uma string de texto: "sigmoid" ou "relu"

    Retorna:
    dA_prev -- Gradiente do custo em relação à ativação (da camada anterior l-1), mesmo formato que A_prev
    dW -- Gradiente do custo em relação a W (camada atual l), mesmo formato que W
    db -- Gradiente do custo em relação a b (camada atual l), mesmo formato que b
    """
    linear_cache, activation_cache = cache

    if activation == "relu":
        ### SEU CÓDIGO COMEÇA AQUI ### ~2 linhas


        ### SEU CÓDIGO TERMINA AQUI ###

    elif activation == "sigmoid":
        ### SEU CÓDIGO COMEÇA AQUI ### ~2 linhas


        ### SEU CÓDIGO TERMINA AQUI ###

    return dA_prev, dW, db

### **5.4: Retropropagação**

A função `linear_activation_backward` calcula a retropropagação para uma única camada $l$. Use esta função para implementar a função `backward_pass`, que realiza a retropropagação dos erros $L(\hat{Y}, Y)$ por todas as camadas da rede, de trás para frente. Esquematicamente, o processo de retropropagação pode ser visto da seguinte forma:

(Linear | ReLU) $\times$ [L-1] $←$ (Linear | Sigmoid) $←$ $L(\hat{Y}, Y)$

In [ ]:
def backward_pass(Y_hat, Y, caches):
    """
    Implemente a propagação regressiva para o grupo [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID

    Argumentos:
    Y_hat -- vetor de probabilidade, saída da propagação direta (L_model_forward())
    Y -- vetor "rótulo" verdadeiro (contendo 0 se não for gato, 1 se for gato)
    caches -- lista de caches contendo:
        todos os caches de linear_activation_forward() com "relu" (há (L-1) ou mais, índices de 0 a L-2)
        o cache de linear_activation_forward() com "sigmoide" (há um, índice L-1)

    Retorna:
    grads -- Um dicionário com os gradientes
    grads["dA" + str(l)] = ...
    grads["dW" + str(l)] = ...
    grads["db" + str(l)] = ...
    """
    grads = {}
    L = len(caches) # número de camadas
    Y = Y.reshape(Y_hat.shape) # após esta linha, Y tem o mesmo formato de AL

    # Calcule d_Yhat para inicializar a retropropagação
    ### SEU CÓDIGO COMEÇA AQUI ### ~1 linha

    ### SEU CÓDIGO TERMINA AQUI ###

    # Gradientes da camada L (SIGMOID -> LINEAR). Entradas: "Y_hat, Y, caches". Saídas: "grads["dAL"], grads["dWL"], grads["dbL"]
    ### SEU CÓDIGO COMEÇA AQUI ### ~2 linhas


    ### SEU CÓDIGO TERMINA AQUI ###

    for l in reversed(range(1, L)):
        # camada l-ésima: gradientes (RELU -> LINEAR).
        ### SEU CÓDIGO COMEÇA AQUI ### ~5 linhas


        ### SEU CÓDIGO TERMINA AQUI ###

    return grads

In [ ]:
np.random.seed(3)
Y_hat = np.random.randn(1, 2); Y = np.array([[1, 0]])

A1 = np.random.randn(4,2); W1 = np.random.randn(3,4); b1 = np.random.randn(3,1); Z1 = np.random.randn(3,2)
linear_cache_activation_1 = ((A1, W1, b1), Z1)

A2 = np.random.randn(3,2); W2 = np.random.randn(1,3); b2 = np.random.randn(1,1); Z2 = np.random.randn(1,2)
linear_cache_activation_2 = ((A2, W2, b2), Z2)

caches = (linear_cache_activation_1, linear_cache_activation_2)

grads = backward_pass(Y_hat, Y, caches)

print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

**Resultados Esperados:**       

<table style="width:60%">
  <tr>
    <td ><b> dW1 </b></td>
           <td > [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]] </td>
  </tr>
    <tr>
    <td > <b> db1 </b></td>
           <td > [[-0.22007063]
 [ 0.        ]
 [-0.02835349]] </td>
  </tr>
  <tr>
  <td > <b> dA1 </b></td>
           <td > [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]] </td>
  </tr>
</table>



### **Parte 6: Otimizar pesos com gradiente descendente**

Antes de implementar o loop principal de gradiente descendente, implemente uma função auxiliar `update_parameters` para atualizar os parâmetros de cada camada $l$ do MLP com seus respectivos gradientes. Lembre-se de que a regra de atualização do gradiente descendente é a seguinte:

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} $$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} $$

onde $\alpha$ é a taxa de aprendizado. Lembre-se de que todos os parâmetros do MLP são armazenados no dicionário `parameters` e os gradientes no dicionário `grads`.

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    """
    Atualiza os parâmetros usando gradiente descendente

    Argumentos:
    parameters -- dicionário Python contendo seus parâmetros
    grads -- dicionário Python contendo seus gradientes, saída de L_model_backward

    Retorna:
    parameters -- dicionário Python contendo seus parâmetros atualizados
        parameters["W" + str(l)] = ...
        parameters["b" + str(l)] = ...
    """

    L = len(parameters) // 2 # números de camadas na rede

    # regra de atualização para cada parâmetro. Use um loop for.
    ### SEU CÓDIGO COMEÇA AQUI ### ~3 linhas



    ### SEU CÓDIGO TERMINA AQUI ###

    return parameters

In [ ]:
np.random.seed(2)
W1 = np.random.randn(3,4); b1 = np.random.randn(3,1)
W2 = np.random.randn(1,3); b2 = np.random.randn(1,1)
parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}

np.random.seed(3)
dW1 = np.random.randn(3,4); db1 = np.random.randn(3,1)
dW2 = np.random.randn(1,3); db2 = np.random.randn(1,1)
grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}

parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

**Resultados Esperados:**       

<table style="width:100%">
    <tr>
    <td > <b> W1 </b></td>
           <td > [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]] </td>
  </tr>
    <tr>
    <td > <b> b1 </b></td>
           <td > [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]] </td>
  </tr>
  <tr>
    <td > <b> W2 </b> </td>
           <td > [[-0.55569196  0.0354055   1.32964895]]</td>
  </tr>
    <tr>
    <td > <b> b2 </b> </td>
           <td > [[-0.84610769]] </td>
  </tr>
</table>


Implemente o loop principal de descida do gradiente usando as seguintes funções implementadas ao longo deste projeto: `initialize_parameters`, `forward_pass`, `binary_cross_entropy`, `backward_pass` e `update_parameters`.

In [ ]:
def optimize(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Implementa uma rede neural de L camadas: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.

    Argumentos:
    X -- dados, array numpy de formato (num_px * num_px * 3, número de exemplos)
    Y -- vetor "rótulo" verdadeiro (contendo 0 se cat, 1 se não cat), de formato (1, número de exemplos)
    layers_dims -- lista contendo o tamanho da entrada e o tamanho de cada camada, de comprimento (número de camadas + 1).
    learning_rate -- taxa de aprendizado da regra de atualização de gradiente descendente
    num_iterations -- número de iterações do loop de otimização
    print_cost -- se True, imprime o custo a cada 100 passos

    Retorna:
    parameters -- parâmetros aprendidos pelo modelo. Eles podem então ser usados ​​para prever.
    """

    np.random.seed(1)
    losses = []                         # mantém o histórico das perdas

    # inicializa os parâmetros
    ### SEU CÓDIGO COMEÇA AQUI ### ~1 linha

    ### SEU CÓDIGO TERMINA AQUI ###

    # laço (descida de gradiente)
    for i in range(0, num_iterations):
        # Propagação para frente: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        ### SEU CÓDIGO COMEÇA AQUI ### ~1 linha

        ### SEU CÓDIGO TERMINA AQUI ###

        # calcula a perda.
        ### SEU CÓDIGO COMEÇA AQUI ### ~1 linha

        ### SEU CÓDIGO TERMINA AQUI ###

        # Retropropagração.
        ### SEU CÓDIGO COMEÇA AQUI ### ~1 linha

        ### SEU CÓDIGO TERMINA AQUI ###

        # atualiza os parâmetros.
        ### SEU CÓDIGO COMEÇA AQUI ### ~1 linha

        ### SEU CÓDIGO TERMINA AQUI ###

        # imprime a função de perda a cada 100 exemplos de treino
        if print_cost and i % 100 == 0:
            print ("Loss após a iteração %i: %f" %(i, loss))
        if print_cost and i % 100 == 0:
            losses.append(loss)

    return parameters, losses

Em seguida, você usará a função `optimize` para treinar um MLP de 2 camadas e traçar sua curva de aprendizado.

In [ ]:
learning_rate = 0.0075
layers_dims = [12288, 7, 1]

parameters_2layers, losses_2layers = optimize(train_set_x, train_set_y, layers_dims, learning_rate, num_iterations = 2500, print_cost = True)

# visualiza o custo
plt.plot(np.squeeze(losses_2layers))
plt.ylabel('custo')
plt.xlabel('iterações (por centena)')
plt.title("Taxa de Aprendizado =" + str(learning_rate))
plt.show()

**Resultados Esperados:**       
<table>
    <tr>
        <td> <b> Custo após iteração 0 </b></td>
        <td> 0.695046 </td>
    </tr>
    <tr>
        <td> <b>Custo após iteração 100 </b></td>
        <td> 0.589260 </td>
    </tr>
    <tr>
        <td> <b>...</b></td>
        <td> ... </td>
    </tr>
    <tr>
        <td> <b> Custo após iteração 2400 </b></td>
        <td> 0.026615 </td>
    </tr>
</table>

Agora use a mesma função `optimize` para treinar um MLP de 4 camadas e traçar sua curva de aprendizado.

In [ ]:
#  modelo com 4-camadas
learning_rate = 0.0075
layers_dims = [12288, 20, 7, 5, 1]

parameters_4layers, losses_4layers = optimize(train_set_x, train_set_y, layers_dims, learning_rate, num_iterations = 2500, print_cost = True)

# visualizar o custo
plt.plot(np.squeeze(losses_4layers))
plt.ylabel('custo')
plt.xlabel('iterações (por centena)')
plt.title("Taxa de Aprendizado =" + str(learning_rate))
plt.show()

**Resultados Esperados:**       
<table>
    <tr>
        <td> <b> Cost after iteration 0 </b></td>
        <td> 0.771749 </td>
    </tr>
    <tr>
        <td> <b> Cost after iteration 100 </b></td>
        <td> 0.672053 </td>
    </tr>
    <tr>
        <td> <b>...</b></td>
        <td> ... </td>
    </tr>
    <tr>
        <td> <b> Cost after iteration 2400</b></td>
        <td> 0.092878 </td>
    </tr>
</table>

### **Parte 7: Avaliar modelos treinados**

Finalmente, aplique o *threshold*   $ \ \hat{y} =
\begin{cases}
1,\ \text{if}\ h(x) \geq 0.5\\
0,\ \text{if}\ h(x) < 0.5\\
\end{cases}$ para classificar todos os exemplos em uma matriz $X$ de exemplos.

In [ ]:
def predict(X, y, parameters):
    """
    Esta função é usada para prever os resultados de uma rede neural de camadas L.

    Argumentos:
    X -- conjunto de dados de exemplos que você gostaria de rotular
    parameters -- parâmetros do modelo treinado

    Retorna:
    p -- previsões para o conjunto de dados X fornecido
    accuracy -- porcentagem de exemplos rotulados corretamente
    """

    m = X.shape[1]
    n = len(parameters) // 2 # número de camadas na rede
    p = np.zeros((1,m))

    # propagação das entradas
    ### SEU CÓDIGO COMEÇA AQUI ### ~1 linha

    ### SEU CÓDIGO TERMINA AQUI ###

    # converte as probabilidade para predições 0/1
    ### SEU CÓDIGO COMEÇA AQUI ### ~1-2 linhas

    ### SEU CÓDIGO TERMINA AQUI ###

    # calcula a acurácia
    ### SEU CÓDIGO COMEÇA AQUI ### ~1 line

    ### SEU CÓDIGO TERMINA AQUI ###
    return accuracy

In [ ]:
accuracy_train_2layers = predict(train_set_x, train_set_y, parameters_2layers)
accuracy_test_2layers = predict(test_set_x, test_set_y, parameters_2layers)

accuracy_train_4layers = predict(train_set_x, train_set_y, parameters_4layers)
accuracy_test_4layers = predict(test_set_x, test_set_y, parameters_4layers)

print("Acurácia do Modelo de 2 camadas no Treino:", accuracy_train_2layers)
print("Acurácia do Modelo de 2 camadas no Teste:", accuracy_test_2layers)
print("Acurácia do Modelo de 4 camadas no Treino:", accuracy_train_4layers)
print("Acurácia do Modelo de 4 camadas no Teste:", accuracy_test_4layers)

**Resultados Esperados:**       
<table>
    <tr>
    <td>
    <b>Acurácia do Modelo de 2 camadas no Treino</b>
    </td>
    <td>
    1.0
    </td>
    </tr>
    <tr>
    <td> <b>Acurácia do Modelo de 2 camadas no Teste </b></td>
    <td> 0.74 </td>
    </tr>
        <tr>
    <td> <b>Acurácia do Modelo de 4 camadas no Treino</b></td>
    <td> 0.9856459330143541 </td>
    </tr>
        <tr>
    <td> <b>Acurácia do Modelo de 4 camadas no Teste</b></td>
    <td> 0.8 </td>
    </tr>
</table>

Parabéns! Seu MLP de 4 camadas teve um desempenho melhor (80%) do que seu MLP de 2 camadas (74%) quando avaliado com seus dados de teste. Este é um bom desempenho!

### **Parte 8: Teste com suas próprias imagens**

Para testar seu modelo com suas próprias imagens, você precisará enviar as imagens que deseja testar para o Google Drive. Em seguida, configure a variável `image_path` na célula abaixo para apontar para o caminho da imagem no seu Google Drive. Observe que o caminho para o diretório raiz do seu Google Drive é `/content/gdrive/MyDrive/`. Portanto, se a sua imagem estiver na raiz do seu Google Drive e se chamar "my_cat.jpg", a variável deverá ser configurada como `image_path = /content/gdrive/MyDrive/my_cat.jpg`.

In [ ]:
from google.colab import drive
# drive.mount('/content/gdrive')

### SEU CÓDIGO COMEÇA AQUI ### ~1 linha
image_path = "/content/gdrive/MyDrive/<caminho_para_imagem>.jpg"
### SEU CÓDIGO TERMINA AQUI ###


# faz um pré-processamento na imagem para estar compatível com
# o formato de entrada do seu algoritmo
with Image.open(image_path) as im:
  low_res_image = np.array(im.resize((num_px, num_px)))

x = low_res_image.reshape((num_px * num_px * 3, 1)) # converte em um dado 1D
x = x / 255 # normaliza os valores

prob_y, _ = forward_pass(x, parameters_4layers)
y = (prob_y > 0.5) * 1

plt.imshow(low_res_image)
print ("y = " + str(np.squeeze(y)) + ", seu algoritmo prediz que é uma imagem de '" + classes[int(np.squeeze(y)),].decode("utf-8")+  "'.")